In [546]:
import re
# the path of the semantic triples predicted by the model
res_path = ""
# Read the file content
with open(res_path, 'r') as file:
    content = file.read()

# Use regex to find all <Step2> section
step2_sections = re.findall(r'<Step2>(.*?)</Step2>', content, re.DOTALL)
step1_sections = re.findall(r'<Step1>(.*?)</Step1>', content, re.DOTALL)

# Store the extracted sections in a list
step2_list = [section.strip().replace("\n", "").replace("    ", "").replace("  ", " ").replace(",[", ", [") for section in step2_sections]
step1_list = [section.strip().replace("\n", "").replace("    ", "").replace("  ", " ").replace(",[", ", [") for section in step1_sections]
# Print the list to verify
step2_list

["[['crm:E22_Man-Made_Object1', 'crm:P108i_was_produced_by', 'crm:E12_Production1'], ['crm:E22_Man-Made_Object1', 'crm:P102_has_title', 'crm:E35_Title1'], ['crm:E12_Production1', 'crm:P14_carried_out_by', 'crm:E21_Person1'], ['crm:E21_Person1', 'crm:P131_is_identified_by', 'crm:E82_Actor_Appellation1'], ['crm:E21_Person1', 'crm:P98i_was_born', 'crm:E67_Birth1'], ['crm:E21_Person1', 'crm:P100i_died_in', 'crm:E69_Death1'], ['crm:E21_Person1', 'crm:P107i_is_current_or_former_member_of', 'crm:E74_Group1'], ['crm:E67_Birth1', 'crm:P4_has_time-span', 'crm:E52_Time-Span1'], ['crm:E69_Death1', 'crm:P4_has_time-span', 'crm:E52_Time-Span2']]",
 "[['crm:E22_Man-Made_Object1', 'crm:P108i_was_produced_by', 'crm:E12_Production1'], ['crm:E22_Man-Made_Object1', 'crm:P102_has_title', 'crm:E35_Title1'], ['crm:E12_Production1', 'crm:P14_carried_out_by', 'crm:E21_Person1'], ['crm:E21_Person1', 'crm:P131_is_identified_by', 'crm:E82_Actor_Appellation1'], ['crm:E21_Person1', 'crm:P98i_was_born', 'crm:E67_Bir

In [547]:
import json
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)
        
def calculate_precision_recall(gold_semantics, predicted_semantics):
    gold_set = set(tuple(item) for item in gold_semantics)
    predicted_set = set(tuple(item) for item in predicted_semantics)
 
    # Calculate true positives, false positives, and false negatives
    true_positives = len(gold_set.intersection(predicted_set))
    false_positives = len(predicted_set - gold_set)
    false_negatives = len(gold_set - predicted_set)
    print(f"Wrong: {str(predicted_set - gold_set)}")

    # Calculate precision and recall
    if len(predicted_set) == 1 and len(gold_set) == 0 and len(list(predicted_set)[0][0]) == 0:
        precision = 1
        recall = 1
    else:
        precision = true_positives / (len(predicted_set)) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / len(gold_set) if (true_positives + false_negatives) > 0 else 0
    return precision, recall

In [548]:
import networkx as nx
def prune_wrong(semantics, sm_type):
    G = nx.DiGraph()
    for trips in semantics:
        if len(trips) == 3:
           G.add_edge(trips[0], trips[2], predicate=trips[1])
    # Function to find leaf nodes in the graph
    Stop_flag = True
    while Stop_flag:
        Stop_flag = False
        nodes_to_check = list(G.nodes())  # Create a copy of the nodes
        for i in nodes_to_check:
            if G.out_degree(i) == 0 and G.in_degree(i) >= 1:
                if i not in sm_type:
                    print(f"prune: {i}")
                    G.remove_node(i)
                    Stop_flag = True
    updated_triples = [[s, G.edges[s, o]['predicate'], o] for s, o in G.edges()]
    return updated_triples

#convert the notation of the semantic triples
def convert_notation(semantics):
    if len(semantics) == 0:
        return semantics
    map_dict = {}
    for i, item in enumerate(semantics):
        if map_dict.get(item[0][:-1]) is None:
            map_dict[item[0][:-1]] = [item[0]]
        else:
            if item[0] not in map_dict[item[0][:-1]]:
                map_dict[item[0][:-1]].append(item[0])
        if map_dict.get(item[2][:-1]) is None:
            map_dict[item[2][:-1]] = [item[2]]
        else:
            if item[2] not in map_dict[item[2][:-1]]:
                map_dict[item[2][:-1]].append(item[2])
    convert_dict = {}
    for key, value in map_dict.items():
        for i, item in enumerate(value):
            if item[-1] != str(i+1):
                convert_dict[item] = item[:-1] + str(i+1)
    for i, item in enumerate(semantics):
        if convert_dict.get(item[0]) is not None:
            semantics[i][0] = convert_dict[item[0]]
        if convert_dict.get(item[2]) is not None:
            semantics[i][2] = convert_dict[item[2]]
    return semantics

In [ ]:
#golden semantic triples folder path
import os
gold_path = ""
gold_list = [os.path.join(gold_path, i) for i in os.listdir(gold_path)]



In [535]:
pre = []
rec = []
for i, j in enumerate(gold_list):  
    gold_semantics = load_json(j)['internal_link_triples']
    predicted_semantics = [i.replace("'", "").split(", ") for i in step2_list[i][2:-2].split("], [")]
    #Addded
    semantic_type = [i.replace("'", "").split(", ") for i in step1_list[i][2:-2].split("], [")]
    semantic_type = [i[0] for i in semantic_type]
    if len(predicted_semantics[0][0]) > 0:
      print(semantic_type)
      predicted_semantics = prune_wrong(predicted_semantics, semantic_type)
      gold_semantics = sorted(gold_semantics, key=lambda x: (x[0], x[2], x[1]))
      predicted_semantics = sorted(predicted_semantics, key=lambda x: (x[0], x[2], x[1]))
      gold_semantics = convert_notation(gold_semantics)
      predicted_semantics = convert_notation(predicted_semantics)
    #Added
    precision, recall = calculate_precision_recall(gold_semantics, predicted_semantics)
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    pre.append(precision)
    rec.append(recall)
print(f"Mean Precision: {sum(pre) / len(pre)}")
print(f"Mean Recall: {sum(rec) / len(rec)}")

['aac-ont:Person1', 'aac-ont:Person1', 'aac-ont:Person1', 'aac-ont:Person1']
Wrong: {('aac-ont:CulturalHeritageObject1', 'dcterms:creator', 'aac-ont:Person1')}
Precision: 0.0
Recall: 0
['aac-ont:CulturalHeritageObject1', 'aac-ont:Person1', 'aac-ont:Person1', 'aac-ont:Person1', 'aac-ont:Person1']
Wrong: {('edm:EuropeanaAggregation1', 'edm:aggregatedCHO', 'aac-ont:CulturalHeritageObject1')}
Precision: 0.5
Recall: 1.0
['aac-ont:CulturalHeritageObject1', 'edm:WebResource1', 'aac-ont:CulturalHeritageObject1', 'aac-ont:CulturalHeritageObject1', 'aac-ont:CulturalHeritageObject1', 'aac-ont:CulturalHeritageObject1', 'foaf:Document1', 'aac-ont:CulturalHeritageObject1', 'aac-ont:Person1', 'aac-ont:Person1', 'aac-ont:Person1', 'aac-ont:Person1']
Wrong: set()
Precision: 1.0
Recall: 1.0
['aac-ont:CulturalHeritageObject1', 'aac-ont:CulturalHeritageObject1', 'aac-ont:Person1', 'skos:Concept1', 'aac-ont:CulturalHeritageObject1', 'aac-ont:CulturalHeritageObject1', 'edm:WebResource1', 'aac-ont:CulturalHe